# **LV 2: Named Entity Recognition (NER)**
---

##  **1. Uvod u Named Entity Recognition**

**Named Entity Recognition (NER)** je zadatak iz područja *obrade prirodnog jezika (NLP)* gdje se iz teksta automatski prepoznaju i označavaju entiteti poput osoba, lokacija, organizacija, datuma itd.

### Najčešće korištene kategorije entiteta:
- **PERSON** – stvarne osobe
- **ORG** – organizacije, tvrtke, institucije
- **GPE** – političke jedinice (države, gradovi)
- **LOC** – lokacije koje nisu političke
- **DATE** – datumi
- **PRODUCT** – proizvodi
- **MONEY** – novčani iznosi

NER omogućuje da iz nestrukturiranog teksta izvučemo **strukturirane informacije**, što se koristi u:
- **analizi vijesti**
- **praćenju financijskih informacija**
- **analizi društvenih mreža**
- **izradi znanstvenih i tehnoloških baza podataka**
- **chatbotovima i tražilicama**

---


## **2. Korištene biblioteke**

U ovoj vježbi koristimo:

### spaCy
Biblioteka za obradu jezika, uključuje:
- pre-trainane NLP modele
- detekciju entiteta, tokenizaciju, lematizaciju...

### pandas
Za tablični prikaz izdvojenih entiteta.

### matplotlib
Za vizualizaciju frekvencija entiteta.

---


##  **3. Inicijalizacija okruženja i osnovni primjeri**

---



In [1]:
import spacy
from spacy.tokens import Span
import pandas as pd
import matplotlib.pyplot as plt

nlp = spacy.load('en_core_web_sm')

In [2]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 44.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Primjer: *Osnovni NER u spaCy*
---
Demonstracija kako spaCy prepoznaje entitete iz jednostavnog teksta.


In [3]:
text_example1 = 'Lionel Messi plays for Inter Miami since 2023.'
doc = nlp(text_example1)
for ent in doc.ents:
    print(ent.text, ent.label_)

Messi PERSON
Inter Miami LOC
2023 DATE


### Primjer : *Filtriranje i prebrojavanje entiteta*

Ekstrakcija entiteta iz teksta koristi **pre-trained NLP model** koji kreira **Doc objekt** s tokenima i prepoznatim entitetima. Entitete je moguće filtrirati po tipovima koji nas zanimaju, prikazati ih s labelama ili vizualizirati te analizirati njihovu učestalost. Ovaj pristup omogućuje strukturiranje nestrukturiranog teksta i pripremu podataka za vizualizacije, izvještaje ili daljnju obradu u NLP aplikacijama.


In [4]:
text_example2 = """
Apple CEO Tim Cook met with European Union officials in Brussels. The meeting focused on digital privacy regulations planned for 2025.
"""

In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")

doc = nlp(text_example2)

wanted = ["PERSON", "ORG", "GPE"]
filtered = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in wanted]

print("Filtered entities:")
for ent, label in filtered:
    print(f"{ent} → {label}")

from collections import Counter

counts = Counter([ent.label_ for ent in doc.ents])
print("\nEntity counts:", counts)


Filtered entities:
Apple → ORG
Tim Cook → PERSON
European Union → ORG
Brussels → GPE

Entity counts: Counter({'ORG': 2, 'PERSON': 1, 'GPE': 1, 'DATE': 1})


### Primjer : *Ručno dodavanje entiteta*


U spaCy-u je moguće ručno dodati entitete u tekst koristeći **Span** i **doc.set_ents()**.
Ovo omogućuje označavanje entiteta koji model možda ne prepoznaje automatski te pripremu i kreiranje podataka za fine-tuning NER modela.



In [6]:
text_example3 = 'FERIT is a technical faculty known for its strong focus on electrical engineering, computer science, and information technologies. The institution provides modern laboratory facilities and supports a wide range of research and development projects. Students at FERIT work with contemporary technologies and practical examples that prepare them for careers in industry and academia. The faculty continually collaborates with local and international partners to ensure that its programs remain relevant and innovative.'

In [7]:
doc = nlp(text_example3)
ferit = Span(doc, 0, 1, label='ORG')
doc.set_ents([ferit])
[(ent.text, ent.label_) for ent in doc.ents]

[('FERIT', 'ORG')]

# **4. Fine-Tuning NER modela**

SpaCy omogućuje prilagođavanje postojećih modela na vlastite entitete. To znači da možemo:

- **Učiti model da prepozna entitete koji nisu u standardnom** (npr. vrste voća, životinje, proizvode).

- Dodavati nove kategorije entiteta (ner.add_label("NEW_ENTITY")).

- **Fine-tunirati** model na vlastitom malom datasetu koristeći Example.from_dict i nlp.update.

Prednosti fine-tuninga:

- Prepoznavanje entiteta specifičnih za vašu domenu

- Bolja točnost na vlastitim podacima

- Mogućnost kombiniranja sa **pre-trained modelima** da se sačuvaju postojeći entite

### Primjer : *Fine-tunning jednostavnog modela*


In [8]:
import spacy
from spacy.training import Example

TRAIN_DATA = [
    ("I ate an apple today.", {"entities": [(9, 14, "FRUIT")]}),
    ("Bananas are yellow.", {"entities": [(0, 7, "FRUIT")]}),
    ("I like to eat a banana.", {"entities": [(16, 22, "FRUIT")]}),
]

TEST_SENTENCE = "I ate an apple and a banana and Bananas."

nlp = spacy.load("en_core_web_md")
ner = nlp.get_pipe("ner")
ner.add_label("FRUIT")

print("=== BEFORE TRAINING ===")
doc_before = nlp(TEST_SENTENCE)
for ent in doc_before.ents:
    print(ent.text, ent.label_)


=== BEFORE TRAINING ===
Bananas GPE


In [9]:
other_pipes = [p for p in nlp.pipe_names if p != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.resume_training()
    for epoch in range(30):
        for text, ann in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, ann)
            nlp.update([example], drop=0.2, sgd=optimizer)

print("=== AFTER TRAINING ===")
doc_after = nlp(TEST_SENTENCE)
for ent in doc_after.ents:
    print(ent.text, ent.label_)

=== AFTER TRAINING ===
apple FRUIT
banana FRUIT
Bananas FRUIT


# **ZADATCI**


# **Zadatak 1**: Izdvajanje entiteta iz teksta
---
### Tekst:
```
The European Space Agency (ESA) plans to launch satellites from Kourou, French Guiana.
Scientists from Cambridge and MIT support the project.
```
### **Ciljevi**:
1. **Pokrenuti NER** nad tekstom.  
2. **Ispisati sve entitete** prepoznate u tekstu.  
3. **Prikazati entitete** u **pandas tablici** za preglednu vizualizaciju

In [11]:
text_task1 = '''The European Space Agency (ESA) plans to launch satellites from Kourou, French Guiana. Scientists from Cambridge and MIT support the project.'''
import pandas as pd

nlp = spacy.load("en_core_web_sm")
doc = nlp(text_task1)

# 3. Ispiši sve entitete
print("Prepoznati entiteti:\n")
for ent in doc.ents:
    print(f"{ent.text:30} | {ent.label_}")

# 4. Pretvori u pandas DataFrame
data = {
    "entitet": [ent.text for ent in doc.ents],
    "vrsta": [ent.label_ for ent in doc.ents],
    "početak": [ent.start_char for ent in doc.ents],
    "kraj": [ent.end_char for ent in doc.ents],
}

df = pd.DataFrame(data)

print("\nPandas tablica entiteta:\n")
print(df)

Prepoznati entiteti:

The European Space Agency      | ORG
ESA                            | ORG
Kourou                         | GPE
French                         | NORP
Guiana                         | PERSON
Cambridge                      | GPE
MIT                            | ORG

Pandas tablica entiteta:

                     entitet   vrsta  početak  kraj
0  The European Space Agency     ORG        0    25
1                        ESA     ORG       27    30
2                     Kourou     GPE       64    70
3                     French    NORP       72    78
4                     Guiana  PERSON       79    85
5                  Cambridge     GPE      103   112
6                        MIT     ORG      117   120


# **Zadatak 2**: Filtriranje entiteta
---


### Tekst:
```
On Monday, Silicon Valley, the heart of the global tech industry, witnessed a major announcement: Jennifer Collins was appointed as the new CEO of NextGen Robotics, a company headquartered in San Francisco. Collins, who previously worked as the Chief Operating Officer at TechNova Inc. in Seattle, is known for her innovative strategies and bold leadership style.

The appointment came after the sudden resignation of former CEO Mark Reynolds, who cited personal reasons for leaving the company. Analysts from Goldman Analytics in New York speculated that Reynolds’ departure might be linked to recent product delays in the company’s flagship AI-driven drones project.

Following the announcement, Collins held a press conference at the NextGen Robotics headquarters, where she emphasized the importance of teamwork and ethical AI development. “Our goal is to make robotics accessible to small businesses, while maintaining transparency and trust with our customers,” she said. Several journalists from The Tech Times and Global Innovations attended the event, asking about upcoming product launches in Europe and Asia.

Meanwhile, competitors such as RoboWorks Ltd. and FutureTech Solutions in Boston expressed interest in collaborating on industry-wide standards for AI safety. Experts believe that Collins’ appointment could accelerate such partnerships.

By the end of the week, stock prices for NextGen Robotics had risen slightly, reflecting investor confidence in the new leadership. Observers noted that while challenges remain, the company’s direction under Collins seems promising.

Conclusion: This change in leadership not only impacts NextGen Robotics but also signals a broader shift in the tech industry, highlighting the growing importance of ethical practices and collaboration in AI development.
```


### **Ciljevi**:
1. **Pokrenuti NER** nad zadanim tekstom.  
2. **Izdvojiti entitete**: osoba,organizacija i lokacija    
3. Uzmi jednu rečenicu iz teksta i prikaži prepoznate entitete pomoću **spaCy-ove vizualizacije**
4. **Usporediti** prepoznate entitete s izvornim tekstom, navesti ispravne i eventualno pogrešno prepoznate entitete te napisati kratak zaključak o **točnosti modela** i mogućim poboljšanjima

In [17]:
import spacy
from spacy import displacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

text = """
On Monday, Silicon Valley, the heart of the global tech industry, witnessed a major announcement: Jennifer Collins was appointed as the new CEO of NextGen Robotics, a company headquartered in San Francisco. Collins, who previously worked as the Chief Operating Officer at TechNova Inc. in Seattle, is known for her innovative strategies and bold leadership style.

The appointment came after the sudden resignation of former CEO Mark Reynolds, who cited personal reasons for leaving the company. Analysts from Goldman Analytics in New York speculated that Reynolds’ departure might be linked to recent product delays in the company’s flagship AI-driven drones project.

Following the announcement, Collins held a press conference at the NextGen Robotics headquarters, where she emphasized the importance of teamwork and ethical AI development. “Our goal is to make robotics accessible to small businesses, while maintaining transparency and trust with our customers,” she said. Several journalists from The Tech Times and Global Innovations attended the event, asking about upcoming product launches in Europe and Asia.

Meanwhile, competitors such as RoboWorks Ltd. and FutureTech Solutions in Boston expressed interest in collaborating on industry-wide standards for AI safety. Experts believe that Collins’ appointment could accelerate such partnerships.

By the end of the week, stock prices for NextGen Robotics had risen slightly, reflecting investor confidence in the new leadership. Observers noted that while challenges remain, the company’s direction under Collins seems promising.

Conclusion: This change in leadership not only impacts NextGen Robotics but also signals a broader shift in the tech industry, highlighting the growing importance of ethical practices and collaboration in AI development.
"""

# -------------------------
# 1. Pokreni NER
# -------------------------
doc = nlp(text)

# -------------------------
# 2. Izdvoj entitete osoba, organizacija, lokacija
# -------------------------
filtered_ents = [
    (ent.text, ent.label_) for ent in doc.ents
    if ent.label_ in ["PERSON", "ORG", "GPE"]
]

df = pd.DataFrame(filtered_ents, columns=["Entitet", "Tip"])
print(df)

# -------------------------
# 3. Vizualizacija entiteta jedne rečenice
# -------------------------
sentence = list(doc.sents)[0]  # prva rečenica
displacy.render(sentence, style="ent", jupyter=True)
mojText = list(doc.sents)[4]
displacy.render(mojText, style="ent", jupyter=True)

# -------------------------
# 4. Analiza ispravnih i pogrešnih entiteta
# -------------------------
recognized = [(ent.text, ent.label_) for ent in doc.ents]

print("\n--- ANALIZA ENTITETA ---\n")
print("Prepoznati entiteti:")
for ent, label in recognized:
    print(f"{ent:25} | {label}")

# Ručna provjera (ispravnost)
correct = []
incorrect = []

for ent in doc.ents:
    # osnovna procjena prema kontekstu teksta
    if ent.text in text:
        # Provjera: je li oznaka razumna?
        if ent.label_ in ["PERSON", "ORG", "GPE", "DATE"]:
            correct.append((ent.text, ent.label_))
        else:
            # Potencijalno kriva klasifikacija
            incorrect.append((ent.text, ent.label_))

print("\nIspravni entiteti:")
for c in correct:
    print(c)

print("\nMogući pogrešni entiteti:")
for i in incorrect:
    print(i)


                 Entitet     Tip
0       Jennifer Collins  PERSON
1       NextGen Robotics     ORG
2          San Francisco     GPE
3                Collins  PERSON
4          TechNova Inc.     ORG
5                Seattle     GPE
6          Mark Reynolds  PERSON
7      Goldman Analytics     ORG
8               New York     GPE
9               Reynolds     ORG
10                    AI     GPE
11               Collins  PERSON
12        The Tech Times     ORG
13    Global Innovations     ORG
14        RoboWorks Ltd.     ORG
15  FutureTech Solutions     ORG
16                Boston     GPE
17                    AI     GPE
18               Collins  PERSON
19      NextGen Robotics     ORG
20               Collins  PERSON
21      NextGen Robotics     ORG
22                    AI     GPE



--- ANALIZA ENTITETA ---

Prepoznati entiteti:
Monday                    | DATE
Silicon Valley            | LOC
Jennifer Collins          | PERSON
NextGen Robotics          | ORG
San Francisco             | GPE
Collins                   | PERSON
TechNova Inc.             | ORG
Seattle                   | GPE
Mark Reynolds             | PERSON
Goldman Analytics         | ORG
New York                  | GPE
Reynolds                  | ORG
AI                        | GPE
Collins                   | PERSON
The Tech Times            | ORG
Global Innovations        | ORG
Europe                    | LOC
Asia                      | LOC
RoboWorks Ltd.            | ORG
FutureTech Solutions      | ORG
Boston                    | GPE
AI                        | GPE
Collins                   | PERSON
the end of the week       | DATE
NextGen Robotics          | ORG
Collins                   | PERSON
NextGen Robotics          | ORG
AI                        | GPE

Ispravni entiteti:
('Monday', 'DATE

# **Zadatak 3**: Kombinacija predefiniranih i ručno dodanih entiteta
---


### **Tekst:**
```
OpenAI developed ChatGPT, a large language model.
Elon Musk and Sam Altman were involved in the project.
The model is used by companies such as Microsoft, Google, and Apple.
It can generate text, summarize articles, and even write code.
The headquarters are in San Francisco, California.
```

### **Ciljevi:**

1. Pokrenuti NER nad zadanim tekstom.

2. Dodati entitet **PRODUCT** za "ChatGPT" u dokument.

3. Izdvojiti entitete oznaka **ORG**, **PERSON**, **GPE** i **PRODUCT**.

4. Spremiti entitete u rječnik prema tipu entiteta (`ent_dict`).

5. Vizualizirati broj entiteta po kategorijama u **bar chart** pomoću pandas i matplotlib.
6. Napisati kratak zaključak o **preciznosti** NER-a i **korisnosti vizualizacije**

Entiteti po kategorijama:

ORG: ['OpenAI', 'Microsoft', 'Google', 'Apple']
PERSON: ['Elon Musk', 'Sam Altman']
GPE: ['San Francisco', 'California']
PRODUCT: ['ChatGPT']


In [18]:
import spacy
from spacy.tokens import Span
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------------
# 1. Učitavanje modela i teksta
# -------------------------------
nlp = spacy.load("en_core_web_sm")

text_task3 = '''OpenAI developed ChatGPT, a large language model.
Elon Musk and Sam Altman were involved in the project.
The model is used by companies such as Microsoft, Google, and Apple.
It can generate text, summarize articles, and even write code.
The headquarters are in San Francisco, California.'''

doc = nlp(text_task3)

# -------------------------------
# 2. Ručno dodavanje entiteta PRODUCT za "ChatGPT"
# -------------------------------
# Pronađi indeks tokena "ChatGPT"
for token in doc:
    if token.text == "ChatGPT":
        start = token.i
        end = token.i + 1
        break

# Stvaranje novog entiteta
new_ent = Span(doc, start, end, label="PRODUCT")
doc.ents = list(doc.ents) + [new_ent]

# -------------------------------
# 3. Izdvoj samo ORG, PERSON, GPE i PRODUCT
# -------------------------------
valid_labels = ["ORG", "PERSON", "GPE", "PRODUCT"]

filtered_ents = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in valid_labels]

# -------------------------------
# 4. Spremanje entiteta u rječnik po kategorijama
# -------------------------------
ent_dict = {label: [] for label in valid_labels}

for ent_text, ent_label in filtered_ents:
    ent_dict[ent_label].append(ent_text)

print("Entiteti po kategorijama:\n")
for k, v in ent_dict.items():
    print(f"{k}: {v}")

# -------------------------------
# 5. Vizualizacija broja entiteta po kategorijama
# -------------------------------
counts = {label: len(ents) for label, ents in ent_dict.items()}
df_counts = pd.DataFrame(counts.items(), columns=["Entity Type", "Count"])

df_counts.plot(kind="bar", x="Entity Type", y="Count", legend=False, color="skyblue")
plt.title("Broj entiteta po kategorijama")
plt.xlabel("Kategorija entiteta")
plt.ylabel("Broj")
plt.tight_layout()
plt.show()


# **Zadatak 4**: Fine-Tuning NER Modela za Prepoznavanje Životinja




U ovom zadatku imate mali **training set** sa rečenicama u kojima se spominju životinje.  
Cilj je **fine-tunirati spaCy NER model** tako da prepoznaje entitete tipa **`ANIMAL`** u tekstu.

Koristite pre-trained model **`en_core_web_md`**.  

U trening setu entiteti su već anotirani (offseti + label).  
Prikazali smo kako model prepoznaje entitete prije treniranja (**BEFORE TRAINING**).

### **Ciljevi**:
1. **Dodati labelu `ANIMAL`** u NER na osnovu zadanog trening seta  
2. **Fine-tunirati model** da nauči prepoznati životinje  
3. **Testirati model** na primjeru rečenice, npr. "The dog chased the cat near my house."  
4. Razmisliti i navesti kako bi se ovaj pristup mogao koristiti u nekoj stvarnoj primjeni  


In [20]:
import spacy
from spacy.training import Example
from spacy import displacy
import random
import matplotlib.pyplot as plt

# 1. Trening podaci – ANIMAL entiteti
TRAIN_DATA = [
    ("The cat sat on the mat.", {"entities": [(4, 7, "ANIMAL")]}),
    ("A dog barked loudly in the street.", {"entities": [(2, 5, "ANIMAL")]}),
    ("I saw an elephant at the zoo.", {"entities": [(8, 16, "ANIMAL")]}),
    ("The tiger was sleeping under a tree.", {"entities": [(4, 9, "ANIMAL")]}),
    ("A small dog chased the cat.", {"entities": [(8, 11, "ANIMAL"), (22, 25, "ANIMAL")]}),
]

TEST_SENTENCE = "The dog chased the cat near my house."

nlp = spacy.load("en_core_web_md")

# Dodaj ANIMAL labelu
ner = nlp.get_pipe("ner")
ner.add_label("ANIMAL")

print("=== BEFORE TRAINING ===")
doc_before = nlp(TEST_SENTENCE)
displacy.render(doc_before, style="ent", jupyter=True)  # nema entiteta - normalno

# 2. Trening
optimizer = nlp.resume_training()

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    for epoch in range(20):   # ispravljeno
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.2, losses=losses)
        print("Losses", losses)

# 3. Testiranje
print("=== AFTER TRAINING ===")
doc_after = nlp(TEST_SENTENCE)
displacy.render(doc_after, style="ent", jupyter=True)


=== BEFORE TRAINING ===


Losses {'ner': np.float32(7.76127)}


/usr/local/lib/python3.12/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "A small dog chased the cat." with entities "[(8, 11, 'ANIMAL'), (22, 25, 'ANIMAL')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I saw an elephant at the zoo." with entities "[(8, 16, 'ANIMAL')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses {'ner': np.float32(6.464331)}
Losses {'ner': np.float32(5.8996806)}
Losses {'ner': np.float32(5.3416605)}
Losses {'ner': np.float32(5.26647)}
Losses {'ner': np.float32(4.928686)}
Losses {'ner': np.float32(3.9814785)}
Losses {'ner': np.float32(2.2334025)}
Losses {'ner': np.float32(0.39730868)}
Losses {'ner': np.float32(0.027141614)}
Losses {'ner': np.float32(0.0030117184)}
Losses {'ner': np.float32(0.00044246894)}
Losses {'ner': np.float32(4.153354e-05)}
Losses {'ner': np.float32(8.5591255e-06)}
Losses {'ner': np.float32(0.0001915675)}
Losses {'ner': np.float32(9.247798e-06)}
Losses {'ner': np.float32(1.003054e-05)}
Losses {'ner': np.float32(2.3463092e-06)}
Losses {'ner': np.float32(1.5970003e-06)}
Losses {'ner': np.float32(4.0669576e-07)}
=== AFTER TRAINING ===
